# 대구 태풍 데이터
> ###### Made by SSJ

#### Period
- 2022.05.17

#### 사용 데이터
태풍 목록 데이터 : https://data.kma.go.kr/data/typhoonData/typList.do?pgmNo=688  
태풍 예보 데이터 : https://data.kma.go.kr/data/typhoonData/typInfoTYList.do?pgmNo=689  

<br>
<details><summary>위도별 경도의 길이표 (예시로 실제로는 사용하지 않은 표)</summary>

|위도(˚)|위도 1도의 길이(㎞)|경도 1도의 길이(㎞)|
|---|---|---|
|0|110.569|111.322|
|5|110.578|110.902|
|10|110.603|109.643|
|15|110.644|117.553|
|20|110.701|114.650|
|25|110.770|100.953|
|30|110.850|96.490|
|<b>*35*</b>|<b>*110.941*</b>|<b>*91.290*</b>|
|40|111.034|85.397|
|45|111.132|78.850|
|50|111.230|71.700|
|55|111.327|63.997|
|60|111.415|55.803|
|65|111.497|47.178|
|70|111.567|38.188|
|75|111.625|28.904|
|80|111.666|19.394|
|85|111.692|9.735|
|90|111.700|0.000|
</details>

## 0. 필요한 패키지 불러오기

In [10]:
from haversine import haversine
import pandas as pd
import glob

# 1. 파일 읽어오기

In [26]:
# glob을 통해 폴더내에 존재하는 csv의 파일명을 all_files에 저장
all_files = glob.glob("./dataset/cyclone/*.csv")

dfs = []
for i in all_files:
    # 각각의 파일명의 데이터 읽어오기
    df = pd.read_csv(i, encoding="cp949")

    # 파일명에 적혀있는 태풍의 이름 추출
    name = i.replace('./dataset/cyclone\FCT_태풍정보_','').replace('.csv','').replace(' (2)','')
    
    # 24,28,72시간 뒤의 예상 데이터 삭제
    df = df[df['Unnamed: 1'] == 0]
    
    # 태풍명을 DF에 추가
    df['name'] = name
    dfs.append(df)

# 불러온 파일들을 전부 합치기
df = pd.concat(dfs, ignore_index=True)
df

,일시,Unnamed: 1,위도(N),경도(E),중심기압,초속(m/s),시속(km/h),강풍반경(km)[예외반경],강도,크기,진행방향,이동속도(km/h),70% 확률 반경(km),name
0,2015년 08월 15일 03시,0,13.0,148.2,1000,18,65,150,약,소형,서북서,16.0,0.0,고니
1,2015년 08월 15일 09시,0,13.0,147.0,990,24,86,150,약,소형,서,22.0,0.0,고니
2,2015년 08월 15일 15시,0,13.5,146.3,990,24,86,170,약,소형,북서,16.0,0.0,고니
3,2015년 08월 15일 21시,0,14.0,145.7,990,24,86,200,약,소형,북서,14.0,0.0,고니
4,2015년 08월 16일 03시,0,14.1,144.9,985,27,97,220,중,소형,서,15.0,0.0,고니
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1584,2014년 08월 10일 09시,0,34.2,134.3,970,36,130,300,강,중형,북동,27.0,0.0,할롱
1585,2014년 08월 10일 15시,0,36.3,135.7,975,34,122,300,강,중형,북북동,44.0,0.0,할롱
1586,2014년 08월 10일 21시,0,38.9,136.7,980,31,112,280,중,소형,북북동,50.0,0.0,할롱
1587,2014년 08월 11일 03시,0,41.2,137.0,985,27,97,200,중,소형,북,43.0,0.0,할롱


## 2. 태풍의 위치 계산

In [59]:
# 대구 위도 경도
daegu = (35.85, 128.56)

index = []
for i in range(1589):
    cyclone = (df.loc[i,'위도(N)'], df.loc[i,'경도(E)'])

    # 대구가 강풍반경 내에 있는지를 확인
    # haversine은 위도, 경도 데이터에서 km거리를 계산해주는 함수
    if haversine(daegu, cyclone) < df.loc[i,'강풍반경(km)[예외반경]']:
        index.append(i)

# 강풍반경 내에 있는 index만 저장
df1 = df.loc[index,:]
df1

,일시,Unnamed: 1,위도(N),경도(E),중심기압,초속(m/s),시속(km/h),강풍반경(km)[예외반경],강도,크기,진행방향,이동속도(km/h),70% 확률 반경(km),name
44,2015년 08월 25일 09시,0,33.9,130.7,960,39,140,300,강,중형,북북동,38.0,0.0,고니
45,2015년 08월 25일 12시,0,34.9,131.0,970,35,126,280,강,소형,북북동,38.0,0.0,고니
46,2015년 08월 25일 15시,0,35.9,131.2,975,32,115,250,중,소형,북북동,38.0,0.0,고니
323,2013년 10월 08일 21시,0,34.5,129.4,980,31,112,190,중,소형,북동,53.0,0.0,다나스
404,2012년 08월 30일 15시,0,35.7,127.8,995,20,72,150,약,소형,북북동,45.0,0.0,덴빈
424,2010년 08월 11일 06시,0,34.6,127.4,990,24,86,180,약,소형,동북동,27.0,0.0,뎬무
425,2010년 08월 11일 09시,0,34.9,128.0,990,24,86,150,약,소형,동북동,22.0,0.0,뎬무
426,2010년 08월 11일 12시,0,35.3,128.8,990,24,86,120,약,소형,동북동,29.0,0.0,뎬무
427,2010년 08월 11일 15시,0,35.6,129.8,990,24,86,120,약,소형,동북동,33.0,0.0,뎬무
537,2019년 09월 07일 06시,0,34.7,124.9,960,39,140,370,강,중형,북,44.0,NaN,링링


## 3. 날짜에 대한 컬럼 정리

In [46]:
# 연도 컬럼 생성
temp = df1['일시'].str.split('년 ')
df1['년'] = temp.str[0]

# 월 컬럼 생성
temp = temp.str[1].str.split('월 ')
df1['월'] = temp.str[0]

# 일 컬럼 생성
temp = temp.str[1].str.split('일 ')
df1['일'] = temp.str[0]

# 시각 걸럼 생성
df1['시'] = temp.str[1].str.replace('시','')

## 4. 니 마음속에 저★장

In [58]:
# 실제로 필요한 컬럼만 저장
df_final = df1.loc[:,['name','년','월','일','시','위도(N)','경도(E)','중심기압','초속(m/s)','시속(km/h)','강풍반경(km)[예외반경]','강도','크기','진행방향','이동속도(km/h)']]
df_final.to_csv('./output/대구 태풍.csv', index=False, encoding='cp949')